In [3]:
import pandas as pd
import time
from datetime import datetime
from pandas import ExcelWriter 
pd.__version__




'1.0.3'

In [10]:
# temp=1588744463872/1000
time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(temp))

NameError: name 'temp' is not defined

In [3]:
temp=1588744479826/1000
time.strftime('%Y-%m-%d %H:%M:%S', time.localtime(temp))

'2020-05-06 13:54:39'

In [4]:
    # Total number of seconds between dates

In [4]:
class JTLReader:
    def __init__(self,jtlPath):
        self.jtlPath=jtlPath
        self.jtlColumns = ['timeStamp', 'elapsed','label','dataType','bytes','URL','allThreads','success','responseCode','responseMessage']
        %time self.df_Jtl=pd.read_csv(self.jtlPath,usecols=self.jtlColumns,dtype={'label':'category'})#dtype={'label':'category'},nrows=195222    
        self.shape=self.df_Jtl.shape
        self.rows=self.shape[0]
        self.rows
        """Conver Epoc time to Actual Time Start"""
        self.df_Jtl['timeStamp'] = pd.to_datetime(self.df_Jtl['timeStamp'],unit='ms')
        self.df_Jtl['timeStamp'] = self.df_Jtl['timeStamp'].astype('datetime64[s]')#reduce precision pandas timestamp dataframe
        self.df_Jtl['timeStamp']=self.df_Jtl['timeStamp'].dt.tz_localize('utc').dt.tz_convert('Singapore')
        self.df_Jtl['timeStamp'] = self.df_Jtl['timeStamp'].astype(str).str[:-6]
        self.df_Jtl['timeStamp'] = pd.to_datetime(self.df_Jtl['timeStamp'])
        self.df_Jtl['timeStamp'] = self.df_Jtl['timeStamp'].astype('datetime64[s]')
        """Conver Epoc time to Actual Time End"""
        self.filt=(pd.isnull(self.df_Jtl['URL'])) #& (pd.isnull(self.df_Jtl['dataType']))
        self.df_jtl_TC=self.df_Jtl.loc[self.filt]
        self.df_jtl_HTTPReqeust=self.df_Jtl.loc[~self.filt]
        
        self.error_filt=(self.df_jtl_HTTPReqeust['success']==False)
        self.df_HttpErrors=self.df_jtl_HTTPReqeust.loc[self.error_filt]
        
        
        
      
        
    """def getTransactionControllerData(self):
        print("Hello shiva")
        filt=(pd.isnull(self.df_Jtl['URL'])) & (pd.isnull(self.df_Jtl['dataType']))
        self.df_jtl_TC=self.df_Jtl.loc[filt]
        return self.df_jtl_TC"""
        
       
        
    def converTimeStamp(self):
        self.df_Jtl['timeStamp'] = pd.to_datetime(self.df_Jtl['timeStamp'],unit='ms')
        self.df_Jtl['timeStamp'] = self.df_Jtl['timeStamp'].astype('datetime64[s]')#reduce precision pandas timestamp dataframe
        self.df_Jtl['timeStamp']=self.df_Jtl['timeStamp'].dt.tz_localize('utc').dt.tz_convert('Singapore')
        self.df_Jtl['timeStamp'] = self.df_Jtl['timeStamp'].astype(str).str[:-6]
        self.df_Jtl['timeStamp'] = pd.to_datetime(self.df_Jtl['timeStamp'])
        self.df_Jtl['timeStamp'] = self.df_Jtl['timeStamp'].astype('datetime64[s]') 
        
    def ms_to_sec(self):
         self.df_Jtl['elapsed']=((self.df_Jtl['elapsed'])/1000)
        
        
    def overAllResponseTimeSummary(self):
        self.df_overAllResponseTimeSummary=self.df_jtl_TC[['timeStamp','elapsed']]
        return self.df_overAllResponseTimeSummary
       
    def listOfErrors(self):
        return self.df_Jtl['responseCode'].value_counts()
    
    def shapeOfDaraFrame(self):
        return self.df_Jtl.shape
    
    def columnType(self,columnName):
        self.columnName=columnName
        return self.df_Jtl[self.columnName].dtype
    
    def getPeriod(self):
        self.startTime=self.df_Jtl['timeStamp'][0]
        self.endRow=self.df_Jtl['timeStamp'].shape[0]
        self.endTime=self.df_Jtl['timeStamp'][self.endRow-1]
        return f'{self.startTime} : {self.endTime}'
    
    def getDuration(self):
        self.startTime= self.df_Jtl['timeStamp'][0]
        self.endRow= self.df_Jtl['timeStamp'].shape[0]
        self.endTime= self.df_Jtl['timeStamp'][self.endRow-1]#-1
        self.executionStartTime=datetime(self.startTime.year,self.startTime.month,self.startTime.day,self.startTime.hour,self.startTime.minute,self.startTime.second)
        self.executionEndTime=datetime(self.endTime.year,self.endTime.month,self.endTime.day,self.endTime.hour,self.endTime.minute,self.endTime.second)
        self.timeDiff = self.executionEndTime-self.executionStartTime
        self.duration_in_s = self.timeDiff.total_seconds()
        self.duration=time.strftime("%H:%M:%S", time.gmtime(self.duration_in_s))
        return self.duration
    
    def getMaxRunningVusers(self):
        self.allThread=self.df_Jtl['allThreads'].max()
        return self.allThread 
    
      
    def getTotalHits(self):
        return self.df_jtl_HTTPReqeust.shape[0]
    
    def getAvgHitsPerSecond(self):
        return self.df_jtl_HTTPReqeust.shape[0]/self.duration_in_s
    
    def getTotalThroughput(self):
        self.totalThroughput=self.df_jtl_TC['bytes'].sum()
        return self.totalThroughput
    
    def getAvgThroughputPerSecond(self):
        self.avgThroughput=(self.df_jtl_TC['bytes'].sum())/self.duration_in_s
        return self.avgThroughput
    
    def getTotalPassFailFreqList(self):
        self.passFail_freq=self.df_jtl_TC['success'].value_counts().to_frame()
        #self.passFailFrequencyeqList=self.passFail_freq.index.tolist()
        
        return self.passFail_freq

        
    
    

    def getAnalysisSummary(self):
        self.period=self.getPeriod()
        self.duration=self.getDuration()
        self.maxRunningVusers=self.getMaxRunningVusers()
        self.totalThroughput=self.getTotalThroughput()
        self.avgThroughputPerSecond=self.getAvgThroughputPerSecond()
        self.totalHits=self.getTotalHits()
        self.avgHitsPerSecond=self.getAvgHitsPerSecond()
        
        self.df_AnalysisSummary=pd.DataFrame(
            {
                'Data':[
                        'Period','Duration','Maximum Running Vusers','Total bytes',
                        'Average bytes/second','TotalHits','Average Hits Per Second',
                        'Total Error','Total Pass','Total Failed'
                ],
                'Values':[
                            self.period,self.duration,self.maxRunningVusers,self.totalThroughput,
                            self.avgThroughputPerSecond,self.totalHits,self.avgHitsPerSecond,
                            'NA',self.getTotalPassFailFreqList(),'NA'
                ]
                
            }
        )
        return self.df_AnalysisSummary
    
    def responseTimeSummary(self):
        self.df_responseTimeSummary=self.df_jtl_TC[['label','elapsed']]
        self.df_txnGroup=self.df_responseTimeSummary.groupby(['label'],observed=True )  #observed=True      
        self.df_txnGroup=self.df_txnGroup['elapsed'].describe(percentiles=[.90])#self.df_txnGroup.describe(percentiles=[.90])
        self.df_txnGroup.columns=self.df_txnGroup.columns.get_level_values(0)#self.df_txnGroup.columns=self.df_txnGroup.columns.get_level_values(1)
        self.df_txnGroup = self.df_txnGroup[['count','min','mean','90%','max',]]
        self.df_txnGroup.rename(columns={'count':'#Samples','min':'Min','mean':'Average','90%':'90 Percentile','max':'Max'},inplace=True)
        ##self.df_txnGroup=self.df_txnErrorGroup.reset_index() """To be enable if you run describe () on data frame. Now we are running on specific column"""
        ##self.df_txnGroup['throughput']=self.df_txnGroup['count']/self.duration_in_s
        ##self.validTxn_filt=(self.df_txnGroup['count']>0)
        self.df_responseTimeSummaryOutput=self.df_txnGroup #self.df_txnGroup[self.validTxn_filt]
       
        return self.df_responseTimeSummaryOutput
    
    def getPassFailSummary(self):
        self.df_passFailSummary=self.df_jtl_TC[['label','success']]
        self.df_passFailGroup=self.df_passFailSummary.groupby(['label'],observed=True)#whe we use observed=True-->refer https://stackoverflow.com/questions/54922327/valueerror-operands-could-not-be-broadcast-together-with-shape-when-calling-pan?answertab=votes#tab-top
        self.df_txnGroup=self.df_passFailGroup['success'].value_counts()
        self.df_txnGroupSlice=self.df_txnGroup.loc[:,]
        self.df_passFailSummaryOutput=self.df_txnGroupSlice.unstack()
        self.df_passFailSummaryOutput=self.df_passFailSummaryOutput.fillna(0)
        self.df_passFailSummaryOutput.rename(columns={True:'Pass',False:'Fail'},inplace=True)
        return self.df_passFailSummaryOutput
        
    def getFinalResponseSummary(self):
        rs=pd.merge(self.df_responseTimeSummaryOutput,self.df_passFailSummaryOutput,left_index=True,right_index=True)
        rs['Error%']=((rs['Fail'])/(rs['#Samples']))*100 
        rs=rs[['#Samples','Pass','Fail','Error%','Min','Average','90 Percentile','Max']]#'Fail','Error%'
        return rs    
    
    def getRunnerVuserGraphData(self):
        '''T=min,S=Seconds'''       
        self.df_vusers=self.df_jtl_TC[['timeStamp','allThreads']]
        self.df_vusers.sort_values(by=['timeStamp'])
        self.df_vusers.set_index('timeStamp',inplace=True)
        self.df_vusers_summary = pd.DataFrame()
        self.df_vusers_summary['Vusers']=self.df_vusers['allThreads'].resample('30T').max()     
        return self.df_vusers_summary
         
        
    
    def getHitsPersecondGraphData(self):
        '''T=min,S=Seconds'''
        #self.df_jtl_HTTPReqeust['hits']=1
        self.df_hits=self.df_jtl_HTTPReqeust[['timeStamp','URL']]#self.df_jtl_HTTPReqeust[['timeStamp','hits']]
        self.df_hits.sort_values(by=['timeStamp'])
        self.df_hits.set_index('timeStamp',inplace=True)
        self.df_hits_summary = pd.DataFrame()
        self.df_hits_summary['hitsCount']=self.df_hits['URL'].resample('30T').count()     
        return self.df_hits_summary
    
    def getThrouhputPersecondGraphData(self):
        '''T=min,S=Seconds'''
        self.df_throughput=self.df_jtl_TC[['timeStamp','bytes']]
        self.df_throughput.sort_values(by=['timeStamp'])
        self.df_throughput.set_index('timeStamp',inplace=True)
        self.df_throughput_summary = pd.DataFrame()
        self.df_throughput_summary['Throughput']=self.df_throughput['bytes'].resample('30T').sum()     
        return self.df_throughput_summary
    
    def getOverallResponseTime(self):
        return self.df_jtl_TC[['timeStamp','elapsed','label']]
    
    def temp(self):
        self.slicer_grp=self.df_jtl_TC[['timeStamp','elapsed','label']]
        self.slicer_grp.sort_values(by=['timeStamp'])
        self.slicer_grp.set_index('timeStamp',inplace=True)
        self.df_response_summary_slicer = pd.DataFrame()
        self.df_slicer_txnGroup=self.slicer_grp.groupby(['label'],observed=True )
        self.df_response_summary_slicer=self.df_slicer_txnGroup['elapsed'].resample('30T').agg(['min','mean','max','count'])
        return self.df_response_summary_slicer
    
    def getResponseMessage(self):
         return self.df_HttpErrors
        
        
    
    
      
obj1=JTLReader("C:\\Karthik\\03_Results\\Temp.jtl") 
#obj1.converTimeStamp()
obj1.ms_to_sec() 
obj1.getDuration()



Wall time: 173 ms


'00:00:46'

In [5]:
y=obj1.getTotalPassFailFreqList()
y

,success
True,3867


In [6]:
x=obj1.temp()
x

,,min,mean,max,count
label,timeStamp,,,,
BP02_eIACSTransactionTime,2020-05-22 14:00:00,14,20.789634,186,1968
BP01_BikeCounterStatus,2020-05-22 14:00:00,14,24.483757,768,1693
BP03_iBorders,2020-05-22 14:00:00,214,218.514563,275,206


In [74]:
#obj1.df_Jtl

In [7]:
obj1.df_jtl_HTTPReqeust

,timeStamp,elapsed,label,responseCode,responseMessage,dataType,success,bytes,allThreads,URL
0,2020-05-22 14:28:52,48,BP01_BikeCounterStatus-0,200,OK,text,True,241,3,http://192.168.161.119/bikes/sib/v1/counterstatus
3,2020-05-22 14:28:52,48,TC02_eIACSTransactionTime-0,200,OK,text,True,239,3,http://192.168.161.119/eiacs/sib/v1/transactio...
4,2020-05-22 14:28:52,21,BP01_BikeCounterStatus-0,200,OK,text,True,241,3,http://192.168.161.119/bikes/sib/v1/counterstatus
7,2020-05-22 14:28:52,21,TC02_eIACSTransactionTime-0,200,OK,text,True,239,3,http://192.168.161.119/eiacs/sib/v1/transactio...
8,2020-05-22 14:28:52,19,BP01_BikeCounterStatus-0,200,OK,text,True,241,3,http://192.168.161.119/bikes/sib/v1/counterstatus
...,...,...,...,...,...,...,...,...,...,...
7725,2020-05-22 14:29:38,19,TC02_eIACSTransactionTime-0,200,OK,text,True,239,3,http://192.168.161.119/eiacs/sib/v1/transactio...
7727,2020-05-22 14:29:38,18,TC02_eIACSTransactionTime-0,200,OK,text,True,239,3,http://192.168.161.119/eiacs/sib/v1/transactio...
7729,2020-05-22 14:29:38,17,TC02_eIACSTransactionTime-0,200,OK,text,True,239,3,http://192.168.161.119/eiacs/sib/v1/transactio...
7731,2020-05-22 14:29:38,25,TC02_eIACSTransactionTime-0,200,OK,text,True,239,3,http://192.168.161.119/eiacs/sib/v1/transactio...


In [8]:
obj1.df_jtl_TC

,timeStamp,elapsed,label,responseCode,responseMessage,dataType,success,bytes,allThreads,URL
1,2020-05-22 14:28:52,48,BP02_eIACSTransactionTime,200,"Number of samples in transaction : 1, number o...",NaN,True,239,3,NaN
2,2020-05-22 14:28:52,48,BP01_BikeCounterStatus,200,"Number of samples in transaction : 2, number o...",NaN,True,241,3,NaN
5,2020-05-22 14:28:52,21,BP02_eIACSTransactionTime,200,"Number of samples in transaction : 1, number o...",NaN,True,239,3,NaN
6,2020-05-22 14:28:52,21,BP01_BikeCounterStatus,200,"Number of samples in transaction : 2, number o...",NaN,True,241,3,NaN
9,2020-05-22 14:28:52,19,BP02_eIACSTransactionTime,200,"Number of samples in transaction : 1, number o...",NaN,True,239,3,NaN
...,...,...,...,...,...,...,...,...,...,...
7724,2020-05-22 14:29:38,19,BP02_eIACSTransactionTime,200,"Number of samples in transaction : 1, number o...",NaN,True,239,3,NaN
7726,2020-05-22 14:29:38,18,BP02_eIACSTransactionTime,200,"Number of samples in transaction : 1, number o...",NaN,True,239,3,NaN
7728,2020-05-22 14:29:38,17,BP02_eIACSTransactionTime,200,"Number of samples in transaction : 1, number o...",NaN,True,239,3,NaN
7730,2020-05-22 14:29:38,25,BP02_eIACSTransactionTime,200,"Number of samples in transaction : 1, number o...",NaN,True,239,3,NaN


In [129]:
#obj1.getPeriod()

In [130]:
#obj1.getDuration()

In [131]:
#obj1.getMaxRunningVusers()

In [132]:
#obj1.getTotalThroughput()

In [19]:
analysisSummary=obj1.getAnalysisSummary()
analysisSummary


,Data,Values
0,Period,2020-05-22 14:28:52 : 2020-05-22 14:29:38
1,Duration,00:00:46
2,Maximum Running Vusers,3
3,Total bytes,926157
4,Average bytes/second,20133.8
5,TotalHits,3867
6,Average Hits Per Second,84.0652
7,Total Error,NA
8,Total Pass,success True 3867
9,Total Failed,NA


In [10]:
#responseTimeSummary
responseTimeSummary=obj1.responseTimeSummary()
responseTimeSummary

,#Samples,Min,Average,90 Percentile,Max
label,,,,,
BP02_eIACSTransactionTime,1968.0,14.0,20.789634,23.0,186.0
BP01_BikeCounterStatus,1693.0,14.0,24.483757,22.0,768.0
BP03_iBorders,206.0,214.0,218.514563,220.5,275.0


In [11]:
passandfail=obj1.getPassFailSummary()
passandfail

success,Pass
label,
BP02_eIACSTransactionTime,1968
BP01_BikeCounterStatus,1693
BP03_iBorders,206


In [12]:
rs=obj1.getFinalResponseSummary()
rs

KeyError: 'Fail'

In [13]:
runningVuser=obj1.getRunnerVuserGraphData()
runningVuser

,Vusers
timeStamp,
2020-05-22 14:00:00,3


In [14]:
hitsPersecond=obj1.getHitsPersecondGraphData() 
hitsPersecond

,hitsCount
timeStamp,
2020-05-22 14:00:00,3867


In [15]:
throughput=obj1.getThrouhputPersecondGraphData()
throughput

,Throughput
timeStamp,
2020-05-22 14:00:00,926157


In [16]:
#overAllResponseTimeSummary
overallResponseTime=obj1.getOverallResponseTime()
overallResponseTime

,timeStamp,elapsed,label
1,2020-05-22 14:28:52,48,BP02_eIACSTransactionTime
2,2020-05-22 14:28:52,48,BP01_BikeCounterStatus
5,2020-05-22 14:28:52,21,BP02_eIACSTransactionTime
6,2020-05-22 14:28:52,21,BP01_BikeCounterStatus
9,2020-05-22 14:28:52,19,BP02_eIACSTransactionTime
...,...,...,...
7724,2020-05-22 14:29:38,19,BP02_eIACSTransactionTime
7726,2020-05-22 14:29:38,18,BP02_eIACSTransactionTime
7728,2020-05-22 14:29:38,17,BP02_eIACSTransactionTime
7730,2020-05-22 14:29:38,25,BP02_eIACSTransactionTime


In [17]:
writer = ExcelWriter('JTL_Log_Analyser_Output.xlsx',mode='w')
#Write Analysis Summary to Excel
analysisSummary.to_excel(writer,sheet_name='AnalysisSummary')
#write response summary to excel with txn name,min,avg,max,90% & max
#rs.to_excel(writer,sheet_name='ResponseSummary')
#Write Running Vusers
runningVuser.to_excel(writer,sheet_name='RunningVuserSummary')
#Write Hits Summary
hitsPersecond.to_excel(writer,sheet_name='HitsSummary')
#Write Throughput Summary
throughput.to_excel(writer,sheet_name='ThroughputSummary')
#Write ResponseTime
overallResponseTime.to_excel(writer,sheet_name='ResponseTimeGraph')
writer.save()







In [94]:
responseMessage=obj1.getResponseMessage()
responseMessage
#responseMessage_grp=responseMessage.groupby('label',observed=True )
#errorSummary=responseMessage_grp['responseMessage'].value_counts()
#errorSummary.unstack()
#responseMessage=responseMessage.set_option('display.max_colwidth',1000)
#x=responseMessage[['label','responseMessage']]
#x.loc[227077,'responseMessage']

#responseMessage['responseMessage']
#'Non HTTP response message: Address already in use: connect'





,timeStamp,elapsed,label,responseCode,responseMessage,dataType,success,bytes,allThreads,URL


In [ ]:
import random
import matplotlib.pyplot as plt
#from matplotlib import style
from matplotlib.pyplot import figure
import matplotlib.dates as mdates
%matplotlib inline


#print(plt.style.available)
#print(plt.__file__)

#style.use('dark_background')
plt.rcParams['figure.figsize']=[25,18]
fig = plt.figure(figsize=(20,10))
fig.set_facecolor('#003366')

# Add suplot syntax

ax1 = fig.add_subplot(211)

#To get data for x and y axis
x,y=Throughput.index,Throughput['Throughput']
#To rotate 
for label in ax1.xaxis.get_ticklabels():
    label.set_rotation(0)

#To print x axis with date and time
xfmt = mdates.DateFormatter('%H:%M:%S')#%d-%m-%y %H:%M:%S//ax.xaxis_date()
ax1.xaxis.set_major_formatter(xfmt)
ax1.fill_between(x,y,color="red",linewidth=5)#https://matplotlib.org/users/dflt_style_changes.html

# x and y label
ax1.set_xlabel('Time',fontsize=20,color='white')#alpha=0.3
ax1.set_ylabel('Throughput/sec',fontsize=20)
ax1.set_title('Throughput',fontsize=20,color='white')

#To drwa 4 lines with the desier colour : Top,Bottom,Left and Right
ax1.spines['bottom'].set_color('black')
ax1.spines['top'].set_color('black')
ax1.spines['left'].set_color('black')
ax1.spines['right'].set_color('black')

#
ax1.xaxis.label.set_color('white')#"Time"
ax1.yaxis.label.set_color('white')#"HitsPerMin"

ax1.tick_params(axis='x', labelcolor='white',labelsize=20)#"%d-%m-%y %H:%M:%S"
ax1.tick_params(axis='y', labelcolor='white',labelsize=20)#"Hits Count: 10,20,30,40,50,60,70"

ax1.set_facecolor('#003366')#https://matplotlib.org/users/colors.html

##003366

ax1.grid(which='major', linestyle='-', linewidth='0.2', color='white')#17becf
#ax1.grid(which='minor', linestyle=':', linewidth='0.5', color='red')

#ax1.set_yticks([0,1,2,3,4,5,6,7,8,9,10])
#ax1.set_xticks(['22:00:00','22:30:00','23:00:00','23:30:00','00:00:00',
               # '00:30:00','01:00:00','01:30:00','02:00:00','02:30:00',
               # '03:00:00','03:30:00','04:00:00','04:30:00'])
#ax.set_ylim((0, 6))
#ax.set_xlim()

#fig.savefig("E:\MaxUser.pdf",dpi=None,facecolor='#003366', edgecolor='#003366',bbox_inches='tight')
plt.show()